In [1]:
import numpy as np
import matplotlib as plt
import nfl_data_py as nfl
import pandas as pd

In [2]:
year = 2020
nfl_2020 = nfl.import_pbp_data([year],  downcast=True, cache=False, alt_path=None)

2020 done.
Downcasting floats.


In [3]:
columns = nfl_2020.columns

In [4]:
#[print(x) for x in columns if 'tac' in x]

In [5]:
team_ids = np.unique( nfl_2020['home_team'].to_list()  )

In [6]:
team_ids

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV',
       'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype='<U3')

In [7]:
assert(len(team_ids)) == 32, 'Post expansion'

In [8]:
seattle_home = nfl_2020[ nfl_2020['home_team'] == 'SEA' ]
seattle_away = nfl_2020[ nfl_2020['away_team'] == 'SEA' ]

we want the tackles done **by** seattle, so seattle can not be the team which possesses the ball

In [9]:
seattle_home = seattle_home[seattle_home['possession_team'] == 'SEA']
seattle_away = seattle_away[seattle_away['possession_team'] == 'SEA']

**home tackles**

In [10]:
solo = len(seattle_home[ seattle_home['solo_tackle'] == 1 ] )
assisted = len(seattle_home[ seattle_home['assist_tackle'] == 1 ] )
seattle_tackles_at_home = solo + assisted

In [11]:
seattle_tackles_at_home

461

**away tackles**

In [12]:
solo = len(seattle_away[ seattle_away['solo_tackle'] ==1 ] )
assisted = len(seattle_away[ seattle_away['assist_tackle'] ==1 ] )
seattle_tackles_away = solo + assisted

In [13]:
seattle_tackles_away

409

:-) less tackles away!!! okay generalize it

In [14]:
def filt_by_team(team, nfl_2020):
    #assert type(team) == str, 'something went wrong with team ' + str(team) + ' which is of team ' + str(type(team))
    home = nfl_2020[ nfl_2020['home_team'] == str(team) ]
    away = nfl_2020[ nfl_2020['away_team'] ==  str(team) ]
    
    home = home[home['possession_team'] == str(team)]
    away = away[away['possession_team'] == str(team)]
    
    return home, away

def total_tackles_home( home ):
    solo = len(home[ home['solo_tackle'] == 1 ] )
    assisted = len(home[ home['assist_tackle'] == 1 ] )
    tackles_at_home = solo + assisted
    return tackles_at_home
    
def total_tackles_away( away ):
    solo = len(away[ away['solo_tackle'] ==1 ] )
    assisted = len(away[away['assist_tackle'] ==1 ] )
    tackles_away = solo + assisted
    return tackles_away


Make dictionary for data where each key is a different team

In [15]:
data = {}
for team in team_ids:
    data[str(team)] = np.zeros((7))

In [16]:
for team in team_ids:
    home, away = filt_by_team(team, nfl_2020)
    
    home_tackles, away_tackles = total_tackles_home( home ), total_tackles_away( away )
    data[team][3], data[team][4] = home_tackles, away_tackles
    # add completions code here please!
    # data[team][5], data[team][6] = home_completions, away_completions  
    

In [17]:
data.keys()

dict_keys(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS'])

In [18]:
# this is done tediously by hand because no data availabe in a nice format
# I guess costume color is not a big priority for football statisticians

data['ARI'][0:3] = (151,35,63)
data['ATL'][0:3] = (167,25,48)
data['BAL'][0:3] = (26,25,95)
data['BUF'][0:3] = (0,51,141)
data['CAR'][0:3] = (0,133,202)
data['CHI'][0:3] = (11,22,42)
data['CIN'][0:3] = (251,79,20)
data['CLE'][0:3] = (49,29,0)
data['DAL'][0:3] = (0,53,148)
data['DEN'][0:3] = (251,79,20)
data['DET'][0:3] = (0,118,182)
data['GB'][0:3] = (24,48,40)
data['HOU'][0:3] = (3,32,47)
data['IND'][0:3] = (0,44,95)
data['JAX'][0:3] = (16,24,32)
data['KC'][0:3] = (227,24,55)
data['LA'][0:3] = (0,53,148)
data['LAC'][0:3] = (0,128,198)
data['LV'][0:3] = (0,0,0)
data['MIA'][0:3] = (0,142,151)
data['MIN'][0:3] = (79,38,131)
data['NE'][0:3] = (0,34,68)
data['NO'][0:3] = (211,188,141)
data['NYG'][0:3] = (1,35,82)
data['NYJ'][0:3] = (18,87,64)
data['PHI'][0:3] = (0,76,84)
data['PIT'][0:3] = (255,182,18)
data['SEA'][0:3] = (0,34,68)
data['SF'][0:3] = (170,0,0)
data['TB'][0:3] = (213,10,10)
data['TEN'][0:3] = (12,35,64)
data['WAS'][0:3] = (90,20,20)

Now let's save this data in a python dict format,

In [22]:
# # uncomment to normalize RGB value to be from (0,1)
# for team in team_ids:
#     data[team][0:3] /= 255
#     print(data[team][0:3])
#     pass

[0.59215686 0.1372549  0.24705882]
[0.65490196 0.09803922 0.18823529]
[0.10196078 0.09803922 0.37254902]
[0.         0.2        0.55294118]
[0.         0.52156863 0.79215686]
[0.04313725 0.08627451 0.16470588]
[0.98431373 0.30980392 0.07843137]
[0.19215686 0.11372549 0.        ]
[0.         0.20784314 0.58039216]
[0.98431373 0.30980392 0.07843137]
[0.         0.4627451  0.71372549]
[0.09411765 0.18823529 0.15686275]
[0.01176471 0.1254902  0.18431373]
[0.         0.17254902 0.37254902]
[0.0627451  0.09411765 0.1254902 ]
[0.89019608 0.09411765 0.21568627]
[0.         0.20784314 0.58039216]
[0.         0.50196078 0.77647059]
[0. 0. 0.]
[0.         0.55686275 0.59215686]
[0.30980392 0.14901961 0.51372549]
[0.         0.13333333 0.26666667]
[0.82745098 0.7372549  0.55294118]
[0.00392157 0.1372549  0.32156863]
[0.07058824 0.34117647 0.25098039]
[0.         0.29803922 0.32941176]
[1.         0.71372549 0.07058824]
[0.         0.13333333 0.26666667]
[0.66666667 0.         0.        ]
[0.835294

In [25]:
data['ARI']

array([5.92156863e-01, 1.37254902e-01, 2.47058824e-01, 4.70000000e+02,
       4.21000000e+02, 0.00000000e+00, 0.00000000e+00])

In [23]:
import pickle

# save dictionary to nfl2020_data.pkl file
with open('nfl2020_data_tackles.pkl', 'wb') as fp:
    pickle.dump(data, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [24]:
with open('nfl2020_data_tackles.pkl', 'rb') as fp:
    data_loaded = pickle.load(fp)
    print('nfl data incoming...')
    print(data_loaded)

nfl data incoming...
{'ARI': array([5.92156863e-01, 1.37254902e-01, 2.47058824e-01, 4.70000000e+02,
       4.21000000e+02, 0.00000000e+00, 0.00000000e+00]), 'ATL': array([6.54901961e-01, 9.80392157e-02, 1.88235294e-01, 4.17000000e+02,
       4.46000000e+02, 0.00000000e+00, 0.00000000e+00]), 'BAL': array([1.01960784e-01, 9.80392157e-02, 3.72549020e-01, 4.00000000e+02,
       5.39000000e+02, 0.00000000e+00, 0.00000000e+00]), 'BUF': array([0.00000000e+00, 2.00000000e-01, 5.52941176e-01, 4.71000000e+02,
       4.64000000e+02, 0.00000000e+00, 0.00000000e+00]), 'CAR': array([  0.        ,   0.52156863,   0.79215686, 380.        ,
       418.        ,   0.        ,   0.        ]), 'CHI': array([4.31372549e-02, 8.62745098e-02, 1.64705882e-01, 4.31000000e+02,
       4.53000000e+02, 0.00000000e+00, 0.00000000e+00]), 'CIN': array([9.84313725e-01, 3.09803922e-01, 7.84313725e-02, 3.99000000e+02,
       4.36000000e+02, 0.00000000e+00, 0.00000000e+00]), 'CLE': array([1.92156863e-01, 1.13725490e-01, 0

In [ ]:
data_loaded['ARI']